In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

tags_url = "https://www.instagram.com/explore/tags/"
my_tag = '전주여행'
url = tags_url + my_tag

DRIVER_DIR = '/home/woomurf/dev/BigData Lab Project/instagramCrawl/chromedriver_linux64/chromedriver'
driver = webdriver.Chrome(DRIVER_DIR)
driver.implicitly_wait(3)

driver.get('chrome://settings/')
driver.execute_script('chrome.settingsPrivate.setDefaultZoom(0.25);')
driver.get(url)

elem = driver.find_element_by_tag_name("body")
link_list = []
num_of_crawling_pages = 10

pagedowns = 1
while pagedowns < num_of_crawling_pages:
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        links = driver.find_elements_by_css_selector('div.v1Nh3 > a')
        for i in links:
            link_list.append(i.get_attribute('href'))
        pagedowns += 1

# 크롤링한 결과를 확인해 봅니다.
print('중복링크 개수', len(link_list))

set_link_list = list(set(link_list))
print('유니크링크 개수', len(set_link_list))

print('유니크링크 구성')
print(set_link_list[:10])

중복링크 개수 633
유니크링크 개수 435
유니크링크 구성
['https://www.instagram.com/p/BsXD7RVHI8H/', 'https://www.instagram.com/p/BsWxwGhhEUb/', 'https://www.instagram.com/p/BsW5tDpF1pZ/', 'https://www.instagram.com/p/BsXcl9blqTl/', 'https://www.instagram.com/p/BsYBVZrAEY3/', 'https://www.instagram.com/p/BsXCPM5l5dI/', 'https://www.instagram.com/p/BsZJ4mKHxVc/', 'https://www.instagram.com/p/BsXSgy7gPIY/', 'https://www.instagram.com/p/BsXAStMgvIu/', 'https://www.instagram.com/p/BsWyOMaAkju/']


In [12]:
first_url = set_link_list[300]
print(first_url)

https://www.instagram.com/p/BsWnDaVhYae/


In [13]:
import requests

req = requests.get(first_url)
html = req.text
header = req.headers
status = req.status_code
is_ok = req.ok

In [14]:
print(html)

<!DOCTYPE html>
<html lang="en" class="no-js not-logged-in client-root">
    <head>
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">

        <title>
톡게스트하우스 전주점 on Instagram: “평일 파티는 내가 요리사!! 인원이 많으면 루프탑파티! 파티신청 인원이 적을때는 휴게실에서 직접 만든음식 함께즐기며 한잔! 언제나 즐겁게 #톡게스트하우스 #01052589477  #전주파티#전주여행 #전주파티게스트하우스 #전주게스트하우스파티…”
</title>

        
        <meta name="robots" content="noimageindex, noarchive">
        <meta name="mobile-web-app-capable" content="yes">
        <meta name="theme-color" content="#000000">
        <meta id="viewport" name="viewport" content="width=device-width, initial-scale=1, minimum-scale=1, maximum-scale=1, viewport-fit=cover">
        <link rel="manifest" href="/data/manifest.json">

        
          <link href="https://graph.instagram.com" rel="preconnect" crossorigin>
        

        <link rel="preload" href="/static/bundles/metro/PostPageContainer.js/b96fe17e17de.js" as="script" type="text/javascript" crossorigin="anon

In [15]:
print(header)

{'Content-Type': 'text/html; charset=utf-8', 'Vary': 'Accept-Language, Cookie, Accept-Encoding', 'Content-Language': 'en', 'Content-Encoding': 'gzip', 'Date': 'Wed, 09 Jan 2019 03:26:30 GMT', 'Strict-Transport-Security': 'max-age=3600', 'Cache-Control': 'private, no-cache, no-store, must-revalidate', 'Pragma': 'no-cache', 'Expires': 'Sat, 01 Jan 2000 00:00:00 GMT', 'X-Frame-Options': 'SAMEORIGIN', 'content-security-policy': "report-uri https://www.instagram.com/security/csp_report/; default-src 'self' https://www.instagram.com; img-src https: data: blob:; font-src https: data:; media-src 'self' blob: https://www.instagram.com https://*.cdninstagram.com https://*.fbcdn.net; manifest-src 'self' https://www.instagram.com; script-src 'self' https://instagram.com https://www.instagram.com https://*.www.instagram.com https://*.cdninstagram.com wss://www.instagram.com https://*.facebook.com https://*.fbcdn.net https://*.facebook.net 'unsafe-inline' 'unsafe-eval' blob:; style-src 'self' https:

In [17]:
print(type(header))

<class 'requests.structures.CaseInsensitiveDict'>


In [82]:
print(is_ok)

True


In [19]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

## 처음에 찾아볼 때, 유저 아이디가 있는 a 태그의 FPmhX 클래스로 들어가려 했으나, JS 로 나중에 동기화 되어 찾을 수 없는듯 하다
# user_id = soup.find_all("a", class_ = "FPmhX") 의 user_id 의 len 은 0이다

## 하지만 다른 한 가지 방법으로 로드하기 이전 metadata 에 해당 게시물의 정보들이 들어있는 듯 하다.
## metadata 에는 본문이 짧게만 나오는 듯 하다
## 전체 본문 내용은 script 를 통해 출력되는 듯 하다. -> 하지만 인코딩이 다른 듯 하다
## display url 을 통해 사진의 링크도 출력된다.
## 결국 BS 로 추출해온 script content 에서 데이터를 뽑아내는게 핵심이다

script_content = soup.find_all("script")

print(script_content)

[<script type="text/javascript">
        (function() {
  var docElement = document.documentElement;
  var classRE = new RegExp('(^|\\s)no-js(\\s|$)');
  var className = docElement.className;
  docElement.className = className.replace(classRE, '$1js$2');
})();
</script>, <script type="text/javascript">
(function() {
  
  if ('PerformanceObserver' in window && 'PerformancePaintTiming' in window) {
    window.__bufferedPerformance = [];
    var ob = new PerformanceObserver(function(e) {
      window.__bufferedPerformance.push.apply(window.__bufferedPerformance,e.getEntries());
    });
    ob.observe({entryTypes:['paint']});
  }
  window.__bufferedErrors = [];
  window.onerror = function(message, url, line, column, error) {
    window.__bufferedErrors.push({
      message: message,
      url: url,
      line: line,
      column: column,
      error: error
    });
    return false;
  };
  window.__initialData = {
    pending: true,
    waiting: []
  };
  function notifyLoaded(item, data) {


In [20]:
meta_content = soup.find_all(property = "instapp:hashtags")
print(meta_content)


[<meta content="전주파티게스트하우스" property="instapp:hashtags"/>, <meta content="여행에미치다" property="instapp:hashtags"/>, <meta content="01052589477" property="instapp:hashtags"/>, <meta content="여행스타그램" property="instapp:hashtags"/>, <meta content="전주게스트하우스파티" property="instapp:hashtags"/>, <meta content="전주게스트하우스" property="instapp:hashtags"/>, <meta content="톡게스트하우스" property="instapp:hashtags"/>, <meta content="전주여행" property="instapp:hashtags"/>, <meta content="전주파티" property="instapp:hashtags"/>]


In [21]:
user_content = soup.find(rel="canonical")
username_pp = user_content.get('href')
print(username_pp)

https://www.instagram.com/junhochoi7771/p/BsWnDaVhYae/


In [22]:
## 추출하고자 하는 내용들
## username ( 유저 아이디) -> OK
## contents ( 내용 ) -> OK
## tags ( 태그들 ) -> OK
## image_links ( 이미지 링크들 ) -> OK
## like_num ( 좋아요 개수 )
## comment_num ( 댓글 개수 )


import re


## 해당 script 전체 내용에서 username 의 내용만 추출하여 사용한다
username_p = re.compile("instagram.com/(.*)/p")
username = username_p.findall(str(username_pp))

## username이 작성자 뿐만 아니라 댓글을 단 사용자까지도 포함한다.
## 그런데 username이 웹에서 나타나는 댓글 순서 ( 작성자 : content, 댓글 : comment ) 순서가 아니라 섞여있어 username[0]이 무조건 작성자가 아니다.
print(username)



['junhochoi7771']


In [23]:
## meta data에서 태그들을 추출

hash_tags_p = re.compile("content=\"(.*?)\"")
hash_tags = hash_tags_p.findall(str(meta_content))

print(hash_tags)

['전주파티게스트하우스', '여행에미치다', '01052589477', '여행스타그램', '전주게스트하우스파티', '전주게스트하우스', '톡게스트하우스', '전주여행', '전주파티']


In [24]:
## 이번에는 contents 추출하는 과정입니다
contents_p = re.compile("\"caption\":\".*?\"")
contents = contents_p.findall(str(script_content))
print(contents)
print()

## 이 과정에서 태그가 함께 추출되므로, 이를 분리하는 과정이 약간 필요할 듯 합니다
## 단순하게 # 을 기준으로 태그를 추출할 경우, 본문에 # 을 썼을 경우 문제가 생길 수 있습니다. 
## 현재로서는 따로 태그가 저장되지는 않는 것으로 보입니다



['"caption":"\\ud3c9\\uc77c \\ud30c\\ud2f0\\ub294 \\ub0b4\\uac00 \\uc694\\ub9ac\\uc0ac!!\\n\\uc778\\uc6d0\\uc774 \\ub9ce\\uc73c\\uba74 \\ub8e8\\ud504\\ud0d1\\ud30c\\ud2f0!\\n\\ud30c\\ud2f0\\uc2e0\\uccad \\uc778\\uc6d0\\uc774 \\uc801\\uc744\\ub54c\\ub294 \\ud734\\uac8c\\uc2e4\\uc5d0\\uc11c \\uc9c1\\uc811 \\ub9cc\\ub4e0\\uc74c\\uc2dd \\ud568\\uaed8\\uc990\\uae30\\uba70 \\ud55c\\uc794!\\n\\uc5b8\\uc81c\\ub098 \\uc990\\uac81\\uac8c #\\ud1a1\\uac8c\\uc2a4\\ud2b8\\ud558\\uc6b0\\uc2a4\\n#01052589477\\n\\n#\\uc804\\uc8fc\\ud30c\\ud2f0#\\uc804\\uc8fc\\uc5ec\\ud589 #\\uc804\\uc8fc\\ud30c\\ud2f0\\uac8c\\uc2a4\\ud2b8\\ud558\\uc6b0\\uc2a4 #\\uc804\\uc8fc\\uac8c\\uc2a4\\ud2b8\\ud558\\uc6b0\\uc2a4\\ud30c\\ud2f0 #\\uc804\\uc8fc\\uac8c\\uc2a4\\ud2b8\\ud558\\uc6b0\\uc2a4 #\\uc5ec\\ud589\\uc5d0\\ubbf8\\uce58\\ub2e4 #\\uc5ec\\ud589\\uc2a4\\ud0c0\\uadf8\\ub7a8"']



In [33]:
# 작성 시간 추출

date_p = re.compile("uploadDate\":\"(.{19})\"")
date = date_p.findall(str(script_content))
print(date)

['2019-01-08T00:22:08']


In [48]:
## 이번에는 이미지 링크 추출 과정입니다
image_links_p = re.compile("\"display_resources\":\[.*?\]")
image_links = image_links_p.findall(str(script_content))
print(image_links)
print()

print(len(image_links))
print()

## 여러개의 이미지 링크들에서 개별 이미지 링크로 변환하는 과정입니다.
first_image_link_p = re.compile("\"src\":\".*?\"")
first_image_link = first_image_link_p.findall(str(image_links[0]))
print(first_image_link[0])
print(first_image_link[1])
print(first_image_link[2])
print(len(first_image_link))
print()

second_image_link_p = re.compile("\"src\":\".*?\"")
second_image_link = second_image_link_p.findall(str(image_links[1]))
print(second_image_link[0])
print(second_image_link[1])
print(second_image_link[2])
print(len(second_image_link))
print()

third_image_link_p = re.compile("\"src\":\".*?\"")
third_image_link = second_image_link_p.findall(str(image_links[2]))
print(third_image_link[0])
print(third_image_link[1])
print(third_image_link[2])
print(len(third_image_link))

## 이미지는 2개였지만, 총 3 종류의 링크가 존재했습니다
## 확인해 보니 3개중 1, 2번째 이미지는 같은 것이며 3번째는 다른 것이었습니다
## 추정해 보면 1번째 이미지는 대표 이미지, 2, 3번째 이미지가 포함된 이미지 2종류라고 생각해볼 수 있습니다
## 또한 각 이미지에 대해 resolution 에 의해 3가지의 다른 형태로 저장된 것을 볼 수 있습니다.
## 1번째는 640, 2번째는 750, 3번째는 원본 크기인것으로 보입니다

## 영상의 경우 이미지는 확인

['"display_resources":[{"src":"https://scontent-icn1-1.cdninstagram.com/vp/997dd480c6c1992d4a4073a6e5e3d0c3/5CB5C0F7/t51.2885-15/sh0.08/e35/p640x640/49290807_312663936021919_5049468802986725361_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com","config_width":640,"config_height":800},{"src":"https://scontent-icn1-1.cdninstagram.com/vp/9aebce305725129d08807239138878d3/5CC29CF7/t51.2885-15/sh0.08/e35/p750x750/49290807_312663936021919_5049468802986725361_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com","config_width":750,"config_height":937},{"src":"https://scontent-icn1-1.cdninstagram.com/vp/fde147f83a308efe84484344a682d759/5CCA8D01/t51.2885-15/e35/49290807_312663936021919_5049468802986725361_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com","config_width":1080,"config_height":1350}]', '"display_resources":[{"src":"https://scontent-icn1-1.cdninstagram.com/vp/997dd480c6c1992d4a4073a6e5e3d0c3/5CB5C0F7/t51.2885-15/sh0.08/e35/p640x640/49290807_312663936021919_5049468802986725361_n.jpg?_nc_ht=scontent

In [49]:
## 이번에는 like_num, comment_num 추출하는 과정입니다
likes_comments_num_p = re.compile("\"description\":\".*?Comments")
likes_comments_num = likes_comments_num_p.findall(str(script_content))
print(likes_comments_num)
print()

['"description":"33 Likes, 2 Comments']



In [1]:
## 이제 해당 내용을 정리하여 JSON 객체로 저장하면 됩니다.

### 1. username 크롤링
### 2. image 크롤링
### 3. caption에서 tags 분리하기
### 4. Location도 할 수 있으면 하기
### 5. 특수문자 처리하기 -> 나중에
### 6. date 크롤링 하기
